In [9]:
import pandas as pd
import os

In [317]:
results_total = pd.DataFrame()
for file in os.listdir():
    if pd.Series(file).str.contains('Results')[0]:
        results = pd.read_csv(file)
        results['file'] = str(file)
        results_total = pd.concat([results_total,results])

model = results_total['Unnamed: 0'].str.split('-',expand = True)
model.columns = ['Variable', 'Model_Info', 'Threshold', 'Execution Time (s)']
model['Execution Time (s)'] = model['Execution Time (s)'].str.split(':',expand=True)[1]
model['Threshold'] = model['Threshold'].str.split(' * ',expand=True)[4]
model['Models'] = model['Model_Info'].str.split('(',expand=True)[0]
model['Contamination'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('contamination=',expand=True)[1].str.split(',',expand=True)[0]
model['n_neighbors'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('n_neighbors=',expand=True)[1].str.split(',',expand=True)[0].str.split(')',expand=True)[0]
model['n_estimators'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('n_estimators=',expand=True)[1].str.split(',',expand=True)[0]

results_total = pd.concat([results_total,model], axis=1)

results_total['Model Type'] = results_total.file.str.split('_',expand=True)[0].str.split('Results',expand=True)[0]

results_total['Test %'] = ((results_total.file.str.split('_',expand=True)[1].str.split('.',expand=True)[0].astype(int)-100)*(-1)).astype(str) + ' %'

results_total = results_total[['Variable','Models', 'Model_Info', 'Model Type', 'Test %','Precision', 'Recall', 'Accuracy', 'F1-score', 'AUC',
       'Train Ratio', 'Number of true anomalies',
       'Number of predicted anomalies', 'True Positives', 'False Positives',
       'True Negatives', 'False Negatives',
       'Threshold', 'Execution Time (s)', 'Contamination',
       'n_neighbors', 'n_estimators']]

In [337]:
results_total

,Variable,Models,Model_Info,Model Type,Test %,Precision,Recall,Accuracy,F1-score,AUC,...,Number of predicted anomalies,True Positives,False Positives,True Negatives,False Negatives,Threshold,Execution Time (s),Contamination,n_neighbors,n_estimators
0,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.001, le...",SemiSupervised,30 %,0.0,0.00,0.98,0.0,0.49,...,1814,0,1814,76718,35,11.95,0.23721981048583984,0.001,10,None
1,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.005, le...",SemiSupervised,30 %,0.0,0.00,0.97,0.0,0.49,...,2102,0,2102,76430,35,9.75,0.24769830703735352,0.005,10,None
2,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.01, lea...",SemiSupervised,30 %,0.0,0.00,0.97,0.0,0.48,...,2366,0,2366,76166,35,5.47,0.23105907440185547,0.01,10,None
3,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.05, lea...",SemiSupervised,30 %,0.0,0.09,0.93,0.0,0.51,...,5506,3,5503,73029,32,0.0,0.22705745697021484,0.05,10,None
4,DailyYield_B (KW),LOF,"LOF(algorithm='auto', contamination=0.1, leaf...",SemiSupervised,30 %,0.0,0.20,0.88,0.0,0.54,...,9764,7,9757,68775,28,0.0,0.213972806930542,0.1,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,AC_Power_B (KW),ECOD,"ECOD(contamination=0.005, n_jobs=1)",Unsupervised,10 %,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,7.0,0.11273884773254395,0.005,None,None
116,AC_Power_B (KW),ECOD,"ECOD(contamination=0.01, n_jobs=1)",Unsupervised,10 %,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,6.21,0.11586785316467285,0.01,None,None
117,AC_Power_B (KW),ECOD,"ECOD(contamination=0.05, n_jobs=1)",Unsupervised,10 %,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,3.87,0.13408303260803223,0.05,None,None
118,AC_Power_B (KW),ECOD,"ECOD(contamination=0.1, n_jobs=1)",Unsupervised,10 %,0.0,0.57,0.91,0.0,0.74,...,2355,4,2351,23831,3,2.89,0.12428545951843262,0.1,None,None


In [342]:
import plotly.express as px
df = results_total.copy()
df = df.reset_index(drop=True)
df = df[df['Variable'] == 'AC_Power_B (KW) ']
df = df.loc[df.groupby(['Variable','Models', 'Model Type','Test %'])['F1-score'].idxmax().values,:].sort_values(by='F1-score', ascending = False)
pd.pivot_table(df, values = ["F1-score","AUC","Accuracy","Precision"], index = ["Model Type","Models"], columns= "Test %")

AUC             Accuracy             F1-score         
Test %                   10 %  20 %  30 %     10 %  20 %  30 %     10 %  20 %   
Model Type     Models                                                           
SemiSupervised  ABOD     0.50  0.50  0.50     1.00  1.00  1.00     0.00  0.00  \
                ECOD     0.76  0.82  0.87     0.96  0.95  0.95     0.01  0.01   
                HBOS     0.79  0.82  0.88     1.00  1.00  1.00     0.73  0.74   
                IForest  0.64  0.66  0.69     1.00  1.00  1.00     0.44  0.44   
                KNN      0.79  0.84  0.89     1.00  1.00  1.00     0.73  0.75   
                LOF      0.64  0.68  0.70     1.00  1.00  1.00     0.44  0.53   
Unsupervised    ABOD     0.50  0.50  0.50     1.00  1.00  1.00     0.00  0.00   
                ECOD     0.74  0.79  0.84     0.91  0.89  0.90     0.00  0.01   
                HBOS     0.64  0.64  0.68     1.00  1.00  1.00     0.44  0.43   
                IForest  0.64  0.66  0.69     1.00  1.00  1.00     0.44  0.44   
                KNN      0.79  0.84  0.89     1.00  1.00  1.00     0.73  0.75   
                LOF      0.64  0.70  0.85     1.00  1.00  1.00     0.33  0.50   

                              Precision              
Test %                   30 %      10 %  20 %  30 %  
Model Type     Models                                
SemiSupervised  ABOD     0.00       0.0  0.00  0.00  
                ECOD     0.02       0.0  0.01  0.01  
                HBOS     0.73       1.0  0.88  0.69  
                IForest  0.47       1.0  0.70  0.65  
                KNN      0.73       1.0  0.83  0.69  
                LOF      0.58       1.0  1.00  1.00  
Unsupervised    ABOD     0.00       0.0  0.00  0.00  
                ECOD     0.01       0.0  0.00  0.01  
                HBOS     0.53       1.0  1.00  1.00  
                IForest  0.42       1.0  0.70  0.48  
                KNN      0.72       1.0  0.83  0.67  
                LOF      0.66       0.4  0.64  0.63

In [344]:
import plotly.express as px
df = results_total.copy()
df = df.reset_index(drop=True)
df = df[df['Variable'] == 'AC_Power_B (KW) ']
df = df.loc[df.groupby(['Variable','Models', 'Model Type','Test %'])['F1-score'].idxmax().values,:].sort_values(by='F1-score', ascending = False)
pd.pivot_table(df, values = ["Number of true anomalies","Number of predicted anomalies","True Positives","False Positives"], index = ["Model Type","Models"], columns= "Test %")

False Positives               
Test %                             10 %  20 %  30 %   
Model Type     Models                                 
SemiSupervised  ABOD                  0     0     0  \
                ECOD               1087  2421  3753   
                HBOS                  0     2    20   
                IForest               0     3    12   
                KNN                   0     3    21   
                LOF                   0     0     0   
Unsupervised    ABOD                  0     0     0   
                ECOD               2351  5678  7777   
                HBOS                  0     0     0   
                IForest               0     3    24   
                KNN                   0     3    23   
                LOF                   3     5    24   

                        Number of predicted anomalies               
Test %                                           10 %  20 %  30 %   
Model Type     Models                                               
SemiSupervised  ABOD                                0     0     0  \
                ECOD                             1091  2436  3799   
                HBOS                                4    16    65   
                IForest                             2    10    34   
                KNN                                 4    18    67   
                LOF                                 2     8    24   
Unsupervised    ABOD                                0     0     0   
                ECOD                             2355  5693  7823   
                HBOS                                2     6    21   
                IForest                             2    10    46   
                KNN                                 4    18    69   
                LOF                                 5    14    65   

                        Number of true anomalies           True Positives   
Test %                                      10 % 20 % 30 %           10 %   
Model Type     Models                                                       
SemiSupervised  ABOD                           7   22   59              0  \
                ECOD                           7   22   59              4   
                HBOS                           7   22   59              4   
                IForest                        7   22   59              2   
                KNN                            7   22   59              4   
                LOF                            7   22   59              2   
Unsupervised    ABOD                           7   22   59              0   
                ECOD                           7   22   59              4   
                HBOS                           7   22   59              2   
                IForest                        7   22   59              2   
                KNN                            7   22   59              4   
                LOF                            7   22   59              2   

                                   
Test %                  20 % 30 %  
Model Type     Models              
SemiSupervised  ABOD       0    0  
                ECOD      15   46  
                HBOS      14   45  
                IForest    7   22  
                KNN       15   46  
                LOF        8   24  
Unsupervised    ABOD       0    0  
                ECOD      15   46  
                HBOS       6   21  
                IForest    7   22  
                KNN       15   46  
                LOF        9   41

In [333]:
import plotly.express as px
df = results_total.copy()
df = df.reset_index(drop=True)
df = df[df['Variable'] == 'AC_Power_B (KW) ']
df = df.loc[df.groupby(['Variable','Models', 'Model Type','Test %'])['F1-score'].idxmax().values,:].sort_values(by='F1-score', ascending = False)
color_map = {'SemiSupervised': 'darkblue', 'Unsupervised': 'cornflowerblue'}  # Add more types as needed
fig = px.bar(df, x="Models", y="F1-score", color="Model Type", barmode="group", facet_col="Test %",width=1000,height=500,color_discrete_map=color_map)
fig.update_layout(font=dict(size=16))

# Add labels to the axes and set the title
# Increase font size for x-axis and y-axis labels
fig.show()

In [335]:
import plotly.express as px
df = results_total.copy()
df = df.reset_index(drop=True)
df = df[df['Variable'] == 'AC_Power_B (KW) ']
df['Execution Time (s)'] = df['Execution Time (s)'].astype(float)
df = df.loc[df.groupby(['Variable','Models', 'Model Type','Test %'])['F1-score'].idxmax().values,:].sort_values(by='F1-score', ascending = False)
color_map = {'SemiSupervised': 'darkblue', 'Unsupervised': 'cornflowerblue'}  # Add more types as needed
fig = px.bar(df, x="Models", y="Execution Time (s)", color="Model Type", barmode="group", facet_col="Test %",width=1000, height=400,color_discrete_map=color_map)
fig.update_layout(font=dict(size=16))
fig.show()

In [211]:
import plotly.graph_objects as go
df[['True Positives', 'False Positives', 'True Negatives',
       'False Negatives']]

,True Positives,False Positives,True Negatives,False Negatives
216,15,3,52353,7
576,15,3,52353,7
228,14,2,52354,8
96,46,21,78487,13
696,4,0,26182,3
336,4,0,26182,3
348,4,0,26182,3
108,45,20,78488,14
456,46,23,78485,13
72,24,0,78508,35


In [216]:
data = {
    "Model A - 100 samples": {
        "Class 1": {"TP": 80, "TN": 70, "FP": 10, "FN": 10},
        "Class 2": {"TP": 65, "TN": 85, "FP": 5, "FN": 5},
    },
    "Model A - 500 samples": {
        "Class 1": {"TP": 95, "TN": 85, "FP": 5, "FN": 5},
        "Class 2": {"TP": 70, "TN": 90, "FP": 0, "FN": 10},
    },
    "Model B - 100 samples": {
        "Class 1": {"TP": 75, "TN": 65, "FP": 15, "FN": 15},
        "Class 2": {"TP": 55, "TN": 75, "FP": 15, "FN": 15},
    },
    # ... add data for other models and sample sizes
}


In [223]:
import plotly.express as px

# Define custom color scales for each class
# colorscales = {
#     "Class 1": px.colors.Greens,
#     "Class 2": px.colors.Blues,
# }

# Create the heatmap
fig = px.heatmap(
    data,
    x=list(data.keys()),
    y=list(data.values()[0].keys()),
    z=list(data.values()[0].values()),
    custom_data=list(data.values()),
    hover_data=["TP", "TN", "FP", "FN"],
)

# Add annotations for individual values
for i, row in enumerate(fig.data):
    for j, col in enumerate(row.x):
        fig.add_annotation(
            x=col,
            y=row.y,
            text=f"TP: {data[col][row.y]['TP']}",
            showarrow=False,
            align="center",
        )

# Customize layout and display the plot
fig.update_layout(
    title="Confusion Matrix Comparison",
    xaxis_title="Model & Training Sample Size",
    yaxis_title="Class",
    legend_title="Metric",
    legend_title_textfont_size=12,
)

fig.show()


AttributeError: module 'plotly.express' has no attribute 'heatmap'